In [1]:
from dash import Dash, dcc, html, dash_table, Input, Output, State, ctx
import pandas as pd
import plotly.express as px
import pathlib

CSV_PATH = "gyro_readings_clean.csv"
CSV_PATH = pathlib.Path(CSV_PATH)


df = pd.read_csv(CSV_PATH)
if 'timestamp' in df.columns:
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    df = df.sort_values('timestamp', na_position='last').reset_index(drop=True)

VALUE_COLS = [c for c in ['gx_dps', 'gy_dps', 'gz_dps'] if c in df.columns]
if not VALUE_COLS:
    num_cols = df.select_dtypes(include='number').columns.tolist()
    VALUE_COLS = num_cols[:3]

app = Dash(__name__)
app.title = "Gyroscope Dashboard"

def controls():
    return html.Div([
        html.Div([
            html.Label("Graph type"),
            dcc.Dropdown(
                id="graph-type",
                options=[
                    {"label": "Line", "value": "line"},
                    {"label": "Scatter", "value": "scatter"},
                    {"label": "Histogram (distribution)", "value": "hist"},
                    {"label": "Box (distribution)", "value": "box"},
                ],
                value="line", clearable=False,
            ),
        ], style={"flex": 1, "minWidth": 220, "marginRight": "12px"}),

        html.Div([
            html.Label("Axes to display"),
            dcc.Dropdown(
                id="axis-choices",
                options=[{"label": col, "value": col} for col in VALUE_COLS],
                value=VALUE_COLS, multi=True
            ),
        ], style={"flex": 2, "minWidth": 260, "marginRight": "12px"}),

        html.Div([
            html.Label("Samples per page (N)"),
            dcc.Input(id="n-samples", type="number", value=600, min=20, step=20, debounce=True),
            html.Div([
                html.Button("Prev", id="prev-btn", n_clicks=0, style={"marginRight": "8px"}),
                html.Button("Next", id="next-btn", n_clicks=0),
            ], style={"marginTop": "8px"}),
        ], style={"flex": 1, "minWidth": 220}),
    ], style={"display": "flex", "flexWrap": "wrap", "alignItems": "flex-end"})

app.layout = html.Div([
    html.H2("Gyroscope Monitoring Dashboard"),
    html.Div(f"Loaded file: {CSV_PATH.name}", style={"color": "#555", "marginBottom": "10px"}),
    controls(),
    dcc.Store(id="window-start", data=0),
    html.Hr(),
    dcc.Graph(id="gyro-graph", figure={}),
    html.Div(id="range-label", style={"marginTop": "6px", "color": "#555"}),
    html.H3("Summary (for currently displayed window)"),
    dash_table.DataTable(
        id="summary-table",
        columns=[{"name": "stat", "id": "stat"}] + [{"name": col, "id": col} for col in VALUE_COLS],
        data=[],
        style_table={"overflowX": "auto"},
        style_cell={"padding": "6px", "minWidth": "80px"},
        style_header={"fontWeight": "bold"},
    ),
], style={"padding": "16px", "fontFamily": "sans-serif"})

@app.callback(
    Output("window-start", "data"),
    Input("prev-btn", "n_clicks"),
    Input("next-btn", "n_clicks"),
    Input("n-samples", "value"),
    State("window-start", "data"),
    prevent_initial_call=True
)
def move_window(prev_clicks, next_clicks, n_samples, start):
    total = len(df)
    if total == 0:
        return 0
    N = int(n_samples or 100)
    start = int(start or 0)

    trig = ctx.triggered_id
    if trig == "prev-btn":
        start = max(0, start - N)
    elif trig == "next-btn":
        start = min(max(0, total - N), start + N)
    else:
        # n-samples changed → clamp to end
        start = min(start, max(0, total - N))
    return start

@app.callback(
    Output("gyro-graph", "figure"),
    Output("summary-table", "columns"),
    Output("summary-table", "data"),
    Output("range-label", "children"),
    Input("graph-type", "value"),
    Input("axis-choices", "value"),
    Input("n-samples", "value"),
    Input("window-start", "data"),
)
def update_outputs(graph_type, axes, n_samples, start):
    axes = axes or VALUE_COLS
    axes = [a for a in axes if a in df.columns]
    if not axes:
        axes = VALUE_COLS

    total = len(df)
    N = int(n_samples or 100)
    start = int(start or 0)
    end = min(total, start + N)
    window = df.iloc[start:end].copy()

    if graph_type in ("line", "scatter"):
        xcol = 'timestamp' if 'timestamp' in window.columns else None
        melted = window.melt(id_vars=[xcol] if xcol else None,
                             value_vars=axes, var_name="axis", value_name="value")
        if xcol is None:
            melted["index"] = melted.index
            xcol = "index"
        if graph_type == "line":
            fig = px.line(melted, x=xcol, y="value", color="axis",
                          labels={"value": "dps", xcol: "time"})
        else:
            fig = px.scatter(melted, x=xcol, y="value", color="axis",
                             labels={"value": "dps", xcol: "time"})
    elif graph_type == "hist":
        melted = window[axes].melt(var_name="axis", value_name="value")
        fig = px.histogram(melted, x="value", color="axis", barmode="overlay",
                           labels={"value": "dps"})
    else:  # "box"
        melted = window[axes].melt(var_name="axis", value_name="value")
        fig = px.box(melted, x="axis", y="value", points="all",
                     labels={"value": "dps"})

    fig.update_layout(margin=dict(l=20, r=20, t=40, b=20), legend_title_text="Axis")

    summary = window[axes].agg(["mean", "std", "min", "max"]).round(3).reset_index()
    summary.rename(columns={"index": "stat"}, inplace=True)
    columns = [{"name": c, "id": c} for c in summary.columns]
    data = summary.to_dict("records")

    if total == 0:
        label = "No data"
    else:
        if 'timestamp' in window.columns and pd.api.types.is_datetime64_any_dtype(window['timestamp']):
            t0 = window['timestamp'].iloc[0]
            t1 = window['timestamp'].iloc[-1]
            label = f"Showing rows {start}–{end-1} of {total} | Time: {t0} → {t1}"
        else:
            label = f"Showing rows {start}–{end-1} of {total}"

    return fig, columns, data, label

if __name__ == "__main__":
    app.run(debug=True, port=8051, jupyter_mode="tab")


Dash app running on http://127.0.0.1:8051/


<IPython.core.display.Javascript object>